In [1]:
!pip install catboost scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.2 MB/s eta 0:00:00


# **The first cell: generating mock data and building the database**

In [9]:
import pandas as pd
import random
import os
import math

try:
    from faker import Faker
except ImportError:
    os.system('pip install faker')
    from faker import Faker

fake = Faker('ar_SA')
NUM_RECORDS = 50000

CITY_COORDS = {
    'الرياض': (24.7136, 46.6753), 'جدة': (21.5433, 39.1728), 'الدمام': (26.4207, 50.0888),
    'مكة المكرمة': (21.3891, 39.8579), 'المدينة المنورة': (24.5247, 39.5692),
    'تبوك': (28.3835, 36.5662), 'أبها': (18.2465, 42.5117),
    'حائل': (27.5114, 41.7208), 'جازان': (16.8894, 42.5706)
}

RIYADH_NEIGHBORHOODS = ['النسيم', 'الملز', 'قرطبة', 'العليا', 'السليمانية', 'النرجس', 'لبن', 'طويق', 'العزيزية', 'الصحافة']
OTHER_CITIES = list(CITY_COORDS.keys())

def get_distance_km(city1, city2):
    if city1 == city2: return 0.0
    lat1, lon1 = CITY_COORDS.get(city1, (24.0, 45.0))
    lat2, lon2 = CITY_COORDS.get(city2, (24.0, 45.0))
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return round(R * c, 2)



def get_logical_relationship(age, gender):
    if age < 18:

        return random.choice(['أب', 'أم', 'أخ', 'أخت'])
    elif 18 <= age < 40:
        opts = ['أب', 'أم', 'أخ', 'أخت', 'صديق']
        opts.append('زوجة' if gender == 'M' else 'زوج')
        return random.choice(opts)
    else:
        opts = ['ابن', 'ابنة', 'أخ', 'أخت', 'حفيد']
        opts.append('زوجة' if gender == 'M' else 'زوج')
        return random.choice(opts)

def determine_label_smart(age, health, hours, malicious_count, relation, distance_km):


    if health == 'زهايمر' and age < 50:
        return 'بيانات مرفوضة'

    if relation in ['ابن', 'ابنة']:
        if age < 35:
            return 'بيانات مرفوضة'

    if relation == 'حفيد':
        if age < 50:
            return 'بيانات مرفوضة'

    if age < 18 and relation in ['زوج', 'زوجة', 'ابن', 'ابنة', 'حفيد']:
        return 'بيانات مرفوضة'

    safe_hours = max(hours, 0.1)
    implied_speed = distance_km / safe_hours
    if implied_speed > 200:
        return 'تقييد أمني'

    if relation == 'أخرى': return 'بيانات مرفوضة'

    if malicious_count > 0:
        return 'تقييد أمني'

    if hours > 3.0:
        return 'إحالة للعمليات'

    is_child = (age <= 12)
    is_vulnerable = (health in ['زهايمر', 'توحد', 'إعاقة ذهنية'])

    if is_child or is_vulnerable:
        return 'بث مجتمعي عاجل'

    return 'إحالة للعمليات'


data = []

for i in range(NUM_RECORDS):
    age = random.randint(2, 90)
    gender = random.choice(['M', 'F'])

    if random.random() < 0.1:
        health = 'زهايمر'
    else:
        if age > 60:
            health = random.choices(['سليم', 'زهايمر', 'أمراض نفسية'], weights=[0.5, 0.4, 0.1])[0]
        else:
            health = random.choices(['سليم', 'توحد', 'إعاقة ذهنية'], weights=[0.8, 0.1, 0.1])[0]

    hours_elapsed = random.uniform(0.1, 12.0)

    missing_hood = random.choice(RIYADH_NEIGHBORHOODS)
    missing_city = 'الرياض'

    if random.random() < 0.85:
        reporter_city = 'الرياض'
    else:
        reporter_city = random.choice(list(CITY_COORDS.keys()))

    distance_km = get_distance_km(missing_city, reporter_city)

    malicious_count = 0
    if random.random() < 0.1: malicious_count = random.randint(1, 5)

    if random.random() < 0.1:
         relation = random.choice(['ابن', 'زوج', 'حفيد', 'أب'])
    else:
         relation = get_logical_relationship(age, gender)

    label = determine_label_smart(age, health, hours_elapsed, malicious_count, relation, distance_km)

    row = {
        'عمر المفقود': age,
        'صلة القرابة': relation,
        'الحالة الصحية': health,
        'موقع الفقدان - الحي': missing_hood,
        'موقع المبلغ': reporter_city,
        'المسافة_كم': distance_km,
        'عدد البلاغات الكاذبة': malicious_count,
        'الساعات_المنقضية': round(hours_elapsed, 2),
        'التصنيف': label
    }
    data.append(row)

df = pd.DataFrame(data)
df.to_csv('missing_persons_smart.csv', index=False, encoding='utf-8-sig', sep=';')
print(f"Done: Generated {len(df)} records with Strict Logic checks.")
print("توزيع التصنيفات:")
print(df['التصنيف'].value_counts())

Done: Generated 50000 records with Strict Logic checks.
توزيع التصنيفات:
التصنيف
إحالة للعمليات    35529
تقييد أمني         6238
بيانات مرفوضة      4701
بث مجتمعي عاجل     3532
Name: count, dtype: int64


# **The second cell: training the AI model (CatBoost)**

In [10]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

try:
    from catboost import CatBoostClassifier
except ImportError:
    os.system('pip install catboost scikit-learn')
    from catboost import CatBoostClassifier

print("📂 جاري قراءة البيانات الذكية...")
df = pd.read_csv('missing_persons_smart.csv', sep=';')

X = df.drop('التصنيف', axis=1)
y = df['التصنيف']

cat_features = ['صلة القرابة', 'الحالة الصحية', 'موقع الفقدان - الحي', 'موقع المبلغ']

print("✂️ تقسيم البيانات...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("🧠 تدريب المودل (Context & Anomaly Aware)...")

model = CatBoostClassifier(
    iterations=1500,
    learning_rate=0.08,
    depth=8,
    loss_function='MultiClass',
    verbose=200
)

model.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_test, y_test))

print("\n📊 النتائج:")
y_pred = model.predict(X_test)
print(f"✅ الدقة: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred))

model.save_model("smart_model_v2.cbm")
print("💾 تم حفظ المودل المطور.")

📂 جاري قراءة البيانات الذكية...
✂️ تقسيم البيانات...
🧠 تدريب المودل (Context & Anomaly Aware)...
0:	learn: 1.1817437	test: 1.1810435	best: 1.1810435 (0)	total: 325ms	remaining: 8m 7s
200:	learn: 0.0126466	test: 0.0110866	best: 0.0110866 (200)	total: 1m 42s	remaining: 11m 4s
400:	learn: 0.0066652	test: 0.0063627	best: 0.0063627 (400)	total: 2m 57s	remaining: 8m 5s
600:	learn: 0.0043633	test: 0.0045866	best: 0.0045866 (600)	total: 4m 8s	remaining: 6m 12s
800:	learn: 0.0030541	test: 0.0035242	best: 0.0035242 (800)	total: 5m 19s	remaining: 4m 38s
1000:	learn: 0.0022908	test: 0.0029771	best: 0.0029771 (1000)	total: 6m 31s	remaining: 3m 14s
1200:	learn: 0.0018312	test: 0.0026729	best: 0.0026729 (1200)	total: 7m 40s	remaining: 1m 54s
1400:	learn: 0.0014889	test: 0.0024323	best: 0.0024321 (1399)	total: 8m 51s	remaining: 37.5s
1499:	learn: 0.0013464	test: 0.0023356	best: 0.0023328 (1495)	total: 9m 24s	remaining: 0us

bestTest = 0.002332829325
bestIteration = 1495

Shrink model to first 1496 ite

# **The third cell: the interactive system interface (analysis and decision-making)**

In [11]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import math
from datetime import datetime, time, timedelta
from catboost import CatBoostClassifier
import pytz

try:
    model = CatBoostClassifier()
    model.load_model("smart_model_v2.cbm")
except:
    print("⚠️ المودل غير موجود، تأكد من تشغيل خلية التدريب أولاً.")

RELATIONS = ['أب', 'أم', 'أخ', 'أخت', 'ابن', 'ابنة', 'زوج', 'زوجة', 'حفيد', 'أخرى']
HEALTH_STATUSES = ['زهايمر', 'توحد', 'إعاقة ذهنية', 'أمراض نفسية', 'سليم']
HOODS = ['النسيم', 'الملز', 'قرطبة', 'العليا', 'السليمانية', 'النرجس', 'طويق']
CITIES_LIST = ['الرياض', 'جدة', 'الدمام', 'مكة المكرمة', 'المدينة المنورة', 'تبوك', 'حائل', 'جازان', 'أبها']

CITY_COORDS = {
    'الرياض': (24.7136, 46.6753), 'جدة': (21.5433, 39.1728), 'الدمام': (26.4207, 50.0888),
    'مكة المكرمة': (21.3891, 39.8579), 'المدينة المنورة': (24.5247, 39.5692),
    'تبوك': (28.3835, 36.5662), 'أبها': (18.2465, 42.5117),
    'حائل': (27.5114, 41.7208), 'جازان': (16.8894, 42.5706)
}

style = {'description_width': 'initial'}

w_age = widgets.IntSlider(value=10, min=1, max=100, description='عمر المفقود:', style=style)
w_relation = widgets.Dropdown(options=RELATIONS, value='ابن', description='صلة القرابة:', style=style)
w_health = widgets.Dropdown(options=HEALTH_STATUSES, value='سليم', description='الحالة الصحية:', style=style)
w_hood = widgets.Dropdown(options=HOODS, description='حي الفقدان (الرياض):', style=style)

w_date = widgets.DatePicker(description='تاريخ الفقد:', value=datetime.now().date(), style=style)
hours_options = [f"{i:02d}" for i in range(24)]
w_hour = widgets.Dropdown(options=hours_options, value='12', description='ساعة:', layout=widgets.Layout(width='80px'), style=style)
minutes_options = [f"{i:02d}" for i in range(0, 60, 5)]
w_minute = widgets.Dropdown(options=minutes_options, value='00', description='دقيقة:', layout=widgets.Layout(width='80px'), style=style)

w_reporter = widgets.Dropdown(options=CITIES_LIST, value='الرياض', description='موقع المُبلغ (GPS):', style=style)
w_false_counts = widgets.IntText(value=0, description='سجل بلاغات كاذبة:', style=style)
btn = widgets.Button(description='تحليل المخاطر (AI Prediction)', button_style='primary', layout=widgets.Layout(width='98%', height='50px'))
out = widgets.Output()

def calculate_time_diff(selected_date, h, m):
    if selected_date is None: return 0.0
    user_input_naive = datetime.combine(selected_date, time(int(h), int(m)))
    try:
        saudi_tz = pytz.timezone('Asia/Riyadh')
        utc_now = datetime.now(pytz.utc)
        saudi_now = utc_now.astimezone(saudi_tz).replace(tzinfo=None)
    except:
        saudi_now = datetime.now() + timedelta(hours=3)

    diff = saudi_now - user_input_naive
    hours = diff.total_seconds() / 3600
    return max(0.0, round(hours, 2))

def get_demo_distance(city1_name, city2_name):
    if city1_name in HOODS: city1_name = 'الرياض'

    if city1_name == city2_name: return 0.0

    lat1, lon1 = CITY_COORDS.get('الرياض', (24.7, 46.6))
    lat2, lon2 = CITY_COORDS.get(city2_name, (24.7, 46.6))

    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return round(R * c, 2)

def on_click(b):
    with out:
        clear_output()
        hours_diff = calculate_time_diff(w_date.value, w_hour.value, w_minute.value)
        dist_km = get_demo_distance(w_hood.value, w_reporter.value)

        input_data = pd.DataFrame({
            'عمر المفقود': [w_age.value],
            'صلة القرابة': [w_relation.value],
            'الحالة الصحية': [w_health.value],
            'موقع الفقدان - الحي': [w_hood.value],
            'موقع المبلغ': [w_reporter.value],
            'المسافة_كم': [dist_km],
            'عدد البلاغات الكاذبة': [w_false_counts.value],
            'الساعات_المنقضية': [hours_diff]
        })

        try:
            final_pred = model.predict(input_data)[0][0]
            probs = model.predict_proba(input_data)[0]
            max_prob = probs.max() * 100

            if final_pred == 'بيانات مرفوضة':
                color, bg, icon = '#721c24', '#f8d7da', '❌'
                msg = "رفض السيستم البلاغ لوجود تناقض منطقي (AI Validation)."
            elif final_pred == 'بث مجتمعي عاجل':
                color, bg, icon = '#155724', '#d4edda', '📢'
                msg = "⚠️ حالة حرجة: تم تفعيل البث للمتطوعين في النطاق الجغرافي."
            elif final_pred == 'تقييد أمني':
                color, bg, icon = '#856404', '#fff3cd', '🛡️'
                msg = "تم تقييد البلاغ للاشتباه (Anomaly Detected) - إحالة للتحقق."
            else:
                color, bg, icon = '#004085', '#cce5ff', '👮'
                msg = "إحالة البلاغ لغرفة العمليات المركزية للمتابعة."

            display(HTML(f"""
            <div style='background-color:{bg}; border-right: 7px solid {color}; padding: 25px; border-radius:10px; color:{color}; font-family: sans-serif;'>
                <h2 style='margin:0 0 10px 0; color: {color};'>{icon} النتيجة: {final_pred}</h2>
                <p style='font-size:1.1em; margin-bottom:15px; color: {color};'>{msg}</p>
                <div style='background:rgba(255,255,255,0.6); padding:10px; border-radius:5px; font-size:0.9em; color:#222;'>
                    📊 <b>تحليلات المودل:</b><br>
                    - نسبة الثقة بالقرار: <b>{max_prob:.1f}%</b><br>
                    - المسافة بين المبلغ والحدث: <b>{dist_km} كم</b> (عامل مؤثر)<br>
                    - الزمن المنقضي: <b>{hours_diff} ساعة</b>
                </div>
            </div>
            """))

        except Exception as e:
            display(HTML(f"<div style='color:red; padding:10px; border:1px solid red;'>خطأ تقني: {e}</div>"))

btn.on_click(on_click)

header = widgets.HTML("<h2 style='color:#2c3e50; text-align:center;'>🚀نظام الإبلاغ الذكي عن المفقودين</h2>")
grid = widgets.VBox([
    header,
    widgets.HBox([w_age, w_relation]),
    widgets.HBox([w_health, w_hood]),
    widgets.HBox([w_date, w_hour, w_minute]),
    widgets.HBox([w_reporter, w_false_counts]),
    widgets.HTML("<hr>"),
    btn,
    out
])
display(grid)